## Web scraping das lojas que vendem produtos moleca

"Imagine que você precise de endereço e telefone de todas lojas que vendam produtos de uma determinada marca. Você entra no site da empresa, e encontra uma sessão que mostra as lojas que vendem os produtos dessa marca. Porém são muitas lojas em várias cidades e estados diferentes, e pesquisar manualmente levaria muito tempo."

Utilizamos então da técnica de webscraping para conseguir todos esses dados.

A página web que encontram os dados esta em [Moleca](https://www.moleca.com.br/onde-encontrar/pt).

#### Imports das bibliotecas utilizadas, inicialização da lista e funções utilizadas

In [1]:
from bs4 import BeautifulSoup

import requests
import json


def get_provinces() -> list:
    '''
    Busca a lista de estados na página.
    '''
    response = requests.get('https://www.moleca.com.br/onde-encontrar/pt').text
    provinces_option = BeautifulSoup(response, 'html.parser').select('#estado > option')
    provinces_list = []
    for provinces in provinces_option:
        if provinces.get('value'):
            provinces_list.append(provinces.get('value'))
    return provinces_list

def get_cities(province: str) -> dict:
    '''
    Busca a lista de cidades de acordo com o estado informado.
    '''
    response = requests.post('https://www.moleca.com.br/acoes/lista-cidades', data={'secao':'onde', 'lang':'pt', 'pais':'BRASIL', 'estado':province, 'cidade':''})
    return json.loads(response.content.decode())

def parse_store(store) -> dict:
    '''
    Pega os dados da loja informada.
    '''
    store_name = store.select('.colLoja')[0].text
    store_address = store.select('.colEndereco')[0].text
    store_phone = store.select('.colTelefone')[0].p.text
    try: 
        store_aditional_phones = store.select('.morePhones')[0].text.replace('\t', '').replace('\n', '')
    except:
        store_aditional_phones = None
    return {'name':store_name, 'address':store_address, 'phone':store_phone, 'aditional_phones':store_aditional_phones}

def get_stores(province: str, city: str) -> list:
    '''
    Busca as lojas de acordo com a cidade e estado informados.
    '''
    response = requests.post('https://www.moleca.com.br/acoes/lista-lojas', data={'secao':'onde', 'lang':'pt', 'pais':'BRASIL', 'estado':province, 'cidade':city.get('label')})
    response = response.content.decode()
    stores_divs = BeautifulSoup(response, 'html.parser').select('.loja')
    stores_list = []
    for store in stores_divs:
        parsed_store = parse_store(store)
        parsed_store['city'] = city.get('label')
        parsed_store['province'] = province
        stores_list.append(parsed_store)
    return stores_list

stores_list = []

### Busca a lista de estados

In [2]:
provinces = get_provinces()

### Busca de cidades e lojas

Para cada estado, busca a lista de cidades que possuem lojas que vendem produtos moleca, e para cada cidade, busca a lista de lojas.

In [3]:
for province in provinces:
    cities = get_cities(province)
    for city in cities:
        stores = get_stores(province, city)
        stores_list = stores_list + stores

#### Quantidade de lojas encontradas

In [4]:
len(stores_list)

18014